### Model Paramters

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd

In [ ]:
# Load pre-trained LLM model and tokenizer (for paraphrasing)
# Appropriate choice required for proper usage
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

c:\Users\anshu_r9i5971\Desktop\Python\NewsGan\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anshu_r9i5971\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

In [ ]:
####################################

# News Paramter
batch_size = 4
rand_shuffle = True

# Training Paramter


In [ ]:
####################################

# dataset class for the News
class NewsDataset(Dataset):
    def __init__(self, data_frame_addr):
        self.news_Doc, self.news_X, self.news_y = [], [], []

        # read the all news and remove label column
        df = pd.read_csv(data_frame_addr)
        df.drop('Label', axis= 1, inplace= True)

        print(df.shape) # Total samples of news
        self.length = df.shape[0]

        self.news_Doc = df['DocText'].to_list()
        self.news_X = df['factual_content'].to_list()
        self.news_y = df['counter_narrative_with_facts'].to_list()

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # Returns DocText, factual_content, and counter narrative
        return (
            self.news_Doc[idx],
            self.news_X[idx],
            self.news_y[idx])
####################################

news_dataset = NewsDataset('gpt_snopes_with_facts.csv')
news_loader = DataLoader(news_dataset, batch_size= batch_size, shuffle= rand_shuffle)

(2430, 4)


In [ ]:
for (Doc, X, y) in news_loader:
    print(Doc)
    print(X)
    print(y)
    break

('Did a Russian Prostitute Say Donald Trump Was Poorly Endowed? A Russian sex worker said Donald Trump was one of her clients and that he has a small penis. On 13 January 2017, the World News Daily Report (WNDR) web site built upon salacious rumors swirling about Donald Trump in an article claiming a Russian sex worker had revealed the President-elect has a small penis: A 22-year sex worker named Ivana Kamensky, claims she was one of the prostitutes who had sexual intercourse with Donald Trump in Moscow in 2011, adding that the American President had the smallest penis she had ever seen. In an interview with the Moscow Daily Herald, the young woman described in great details the night that she allegedly spent in Mr Trump\'s hotel room five years ago[, claiming] she was hired by Donald Trump along with two of her friends to perform several degrading sex acts, like urinating on him and on each other. She said that she had been shocked by the incredibly small size of his penis, and rememb